<a href="https://colab.research.google.com/github/Ebhibhebhi/image_classifiers/blob/main/image_classifier_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Kaggle dataset

!unzip -q "/content/archive.zip" -d "/content/archive"

In [ ]:
!ls /content/archive

Fish-vs-Cats


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import shutil
import os

SRC_BASE = "/content/archive/Fish-vs-Cats"
DST_BASE = "/content/drive/MyDrive/Colab Notebooks/Programming PyTorch for DL/Fish-vs-Cats"

os.makedirs(DST_BASE, exist_ok=True)

for split in ["train", "val", "test"]:
    src = os.path.join(SRC_BASE, split)
    dst = os.path.join(DST_BASE, split)

    if os.path.exists(src):
        shutil.copytree(src, dst, dirs_exist_ok=True)
        print(f"Copied {split} to Drive")
    else:
        print(f"Missing: {src}")


Copied train to Drive
Copied val to Drive
Copied test to Drive


# Removing corrupted files

In [ ]:
import os
from PIL import Image

ROOT = "/content/archive/Fish-vs-Cats"
splits = ["train", "val", "test"]
classes = ["cat", "fish"]

bad_files = []

# 1) Scan and collect bad images
for split in splits:
    for cls in classes:
        folder = os.path.join(ROOT, split, cls)
        for fname in os.listdir(folder):
            path = os.path.join(folder, fname)
            try:
                with Image.open(path) as img:
                    img.verify()  # checks image integrity
            except Exception:
                bad_files.append(path)

# 2) Remove bad images
for path in bad_files:
    os.remove(path)

print(f"Removed {len(bad_files)} corrupted images.\n")

# 3) Count remaining images
print("Final dataset counts:")
total = 0
for split in splits:
    split_total = 0
    print(f"\n{split.upper()}:")
    for cls in classes:
        folder = os.path.join(ROOT, split, cls)
        count = len([
            f for f in os.listdir(folder)
            if os.path.isfile(os.path.join(folder, f))
        ])
        split_total += count
        print(f"  {cls}: {count}")
    total += split_total
    print(f"  TOTAL: {split_total}")

print(f"\nOVERALL TOTAL IMAGES: {total}")

Removed 19 corrupted images.

Final dataset counts:

TRAIN:
  cat: 417
  fish: 384
  TOTAL: 801

VAL:
  cat: 87
  fish: 22
  TOTAL: 109

TEST:
  cat: 91
  fish: 69
  TOTAL: 160

OVERALL TOTAL IMAGES: 1070


# Balancing data

In [ ]:
import os
import random

ROOT = "/content/archive/Fish-vs-Cats"
splits = ["train", "val", "test"]
classes = ["cat", "fish"]

SEED = 42
DRY_RUN = False  # set to False to actually delete files

random.seed(SEED)

def list_files(folder):
    return [
        f for f in os.listdir(folder)
        if os.path.isfile(os.path.join(folder, f))
    ]

def count_split(split):
    counts = {}
    for cls in classes:
        folder = os.path.join(ROOT, split, cls)
        counts[cls] = len(list_files(folder))
    return counts

print("BEFORE BALANCING:")
before_counts = {}
for split in splits:
    c = count_split(split)
    before_counts[split] = c
    print(f"{split.upper()}: cat={c['cat']} fish={c['fish']} total={c['cat'] + c['fish']}")

print("\nBALANCING PLAN (undersample to the min per split):")
to_delete = []

for split in splits:
    split_counts = before_counts[split]
    target = min(split_counts["cat"], split_counts["fish"])
    print(f"\n{split.upper()}: target per class = {target}")

    for cls in classes:
        folder = os.path.join(ROOT, split, cls)
        files = list_files(folder)
        extra = len(files) - target

        if extra > 0:
            # randomly pick extra files to delete
            delete_files = random.sample(files, extra)
            delete_paths = [os.path.join(folder, f) for f in delete_files]
            to_delete.extend(delete_paths)
            print(f"  {cls}: deleting {extra} files")
        else:
            print(f"  {cls}: deleting 0 files")

print(f"\nTOTAL FILES MARKED FOR DELETION: {len(to_delete)}")

if DRY_RUN:
    print("\nDRY_RUN=True so nothing was deleted.")
    print("If this looks right, set DRY_RUN=False and rerun.")
else:
    deleted = 0
    for path in to_delete:
        if os.path.exists(path):
            os.remove(path)
            deleted += 1
    print(f"\nDeleted {deleted} files.")

    print("\nAFTER BALANCING:")
    total_after = 0
    for split in splits:
        c = count_split(split)
        total_after += c["cat"] + c["fish"]
        print(f"{split.upper()}: cat={c['cat']} fish={c['fish']} total={c['cat'] + c['fish']}")
    print(f"\nOVERALL TOTAL IMAGES AFTER: {total_after}")

BEFORE BALANCING:
TRAIN: cat=417 fish=384 total=801
VAL: cat=87 fish=22 total=109
TEST: cat=91 fish=69 total=160

BALANCING PLAN (undersample to the min per split):

TRAIN: target per class = 384
  cat: deleting 33 files
  fish: deleting 0 files

VAL: target per class = 22
  cat: deleting 65 files
  fish: deleting 0 files

TEST: target per class = 69
  cat: deleting 22 files
  fish: deleting 0 files

TOTAL FILES MARKED FOR DELETION: 120

Deleted 120 files.

AFTER BALANCING:
TRAIN: cat=384 fish=384 total=768
VAL: cat=22 fish=22 total=44
TEST: cat=69 fish=69 total=138

OVERALL TOTAL IMAGES AFTER: 950


# **Preparing Datasets**

In [ ]:
import torchvision
from torchvision import transforms

ROOT = "/content/archive/Fish-vs-Cats"

# Setting up training data
train_data_path = f"{ROOT}/train"

# what transform does is exactly what it sounds like, it is a machine that applies transformations to image data
# When defining the object, you're basically choosing what transformations you want to apply to the data
transform = transforms.Compose([transforms.Resize((64, 64)),
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225])
                                ])
# Here you are now actually creating the transformed dataset
train_data = torchvision.datasets.ImageFolder(root=train_data_path, transform=transform)

# Setting up validation data (compare accuracy against this set after each epoch)

val_data_path = f"{ROOT}/val"
val_data = torchvision.datasets.ImageFolder(root=val_data_path, transform=transform)

# Setting up test data
test_data_path = f"{ROOT}/test"
test_data = torchvision.datasets.ImageFolder(root=test_data_path, transform=transform)

# **Data Loaders**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from PIL import Image, ImageFile

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64
# These dataloaders will pass the data into our neural network
train_data_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val_data, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_data, batch_size=batch_size)

# **Network**

In [ ]:
class SimpleNet(nn.Module):

    def __init__(self):
        super(SimpleNet, self).__init__()
        self.fc1 = nn.Linear(12288, 1000)
        self.fc2 = nn.Linear(1000, 100)
        self.fc3 = nn.Linear(100, 2)

    def forward(self, x):
        x = x.view(-1, 12288) # Converts our 3-D tensor into a 1-D tensor so that it can be fed to the first linear layer
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
simplenet = SimpleNet()

# **Optimizer**

In [ ]:
optimizer = optim.Adam(simplenet.parameters(), lr=0.001)

# **Training**

In [ ]:
from torch._higher_order_ops import out_dtype
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

simplenet.to(device)

def train(model, optimizer, loss_fn, train_loader, val_loader, epochs=20, device="cpu"):
    for epoch in range(1, epochs+1):
        training_loss = 0.0
        valid_loss = 0.0
        model.train()
        for batch in train_loader:
            optimizer.zero_grad()
            inputs, target = batch
            inputs = inputs.to(device)
            target = target.to(device)
            output = model(inputs)
            loss = loss_fn(output, target)
            loss.backward()
            optimizer.step()
            training_loss += loss.data.item() * inputs.size(0)
        training_loss /= len(train_loader.dataset)

        model.eval()
        num_correct = 0
        num_examples = 0
        for batch in val_loader:
            inputs, target = batch
            inputs = inputs.to(device)
            output = model(inputs)
            target = target.to(device)
            loss = loss_fn(output, target)
            valid_loss += loss.data.item() *  inputs.size(0)
            correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], target).view(-1)

            num_correct += torch.sum(correct).item()
            num_examples += correct.shape[0]
        valid_loss /= len(val_loader.dataset)

        print("Epoch: {}, Training Loss: {:.2f}, Validation Loss: {:.2f}, accuracy = {:.2f}".format(epoch, training_loss, valid_loss, num_correct / num_examples))

In [ ]:
train(simplenet,
      optimizer,
      torch.nn.CrossEntropyLoss(),
      train_data_loader,
      val_data_loader,
      epochs=100,
      device=device)

/tmp/ipython-input-3371257371.py:36: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  correct = torch.eq(torch.max(F.softmax(output), dim=1)[1], target).view(-1)


Epoch: 1, Training Loss: 1.29, Validation Loss: 0.90, accuracy = 0.60
Epoch: 2, Training Loss: 0.55, Validation Loss: 0.52, accuracy = 0.72
Epoch: 3, Training Loss: 0.43, Validation Loss: 0.68, accuracy = 0.77
Epoch: 4, Training Loss: 0.32, Validation Loss: 0.59, accuracy = 0.81
Epoch: 5, Training Loss: 0.29, Validation Loss: 0.48, accuracy = 0.79
Epoch: 6, Training Loss: 0.23, Validation Loss: 0.55, accuracy = 0.77
Epoch: 7, Training Loss: 0.21, Validation Loss: 0.49, accuracy = 0.74
Epoch: 8, Training Loss: 0.14, Validation Loss: 0.51, accuracy = 0.77
Epoch: 9, Training Loss: 0.12, Validation Loss: 0.53, accuracy = 0.74
Epoch: 10, Training Loss: 0.08, Validation Loss: 0.88, accuracy = 0.74
Epoch: 11, Training Loss: 0.06, Validation Loss: 0.70, accuracy = 0.74
Epoch: 12, Training Loss: 0.04, Validation Loss: 0.78, accuracy = 0.74
Epoch: 13, Training Loss: 0.03, Validation Loss: 0.74, accuracy = 0.74
Epoch: 14, Training Loss: 0.02, Validation Loss: 0.80, accuracy = 0.74
Epoch: 15, Trai

# **Making Predictions**

In [ ]:
from PIL import Image

labels = ['cat', 'fish']

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

img = Image.open("/content/archive/Fish-vs-Cats/test/fish/238656308_45fef30371.jpg")
img = transform(img)
img = img.unsqueeze(0).to(device)
# the unsqueeze adds a batch dimension. So if the initial shape of our image was:
# (3, 64, 64)
# unsqueezing made it (1, 3, 64, 64)
# because our model expects batches, even if it is of size one

prediction = simplenet(img) # Output is raw logits, just whatever tensor the NN spews out
prediction = prediction.argmax() # finds the index of the largest logit
print(labels[prediction])

cat


# **Saving the model**

In [ ]:
# Saving the model in pickle format:
# torch.save(simplenet, "/tmp/simplenet")

# and we can reload the model as:
# simplenet = torch.load("/tmp/simplenet")

# This stores both the parameters and the structure of the model to a file, but this may be a problem
# if you change the structure of the model at a later time

# For this reason it's more common to save a model's state_dict instead, which is a standard Python dict
# which contains the maps of each layers parameters in the model:

# torch.save(model.state_dict(), PATH)

# To restore, create an instance of the model and then use load_state_dict

# simplenet = SimpleNet()
# simplenet_state_dict = torch.load("/tmp/simplenet")
# simplenet.load_state_dict(simplenet_state_dict)